<a href="https://colab.research.google.com/github/ramprakashyadav/OCR/blob/main/redbull/fijutsu_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 5.6 MB 23.6 MB/s 
     |████████████████████████████████| 4.3 MB 69.7 MB/s 
     |████████████████████████████████| 139 kB 66.3 MB/s 
     |████████████████████████████████| 3.6 MB 64.4 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.6.0-py3-none-any.whl size=33688 sha256=9a51ff5dc448f5eaa1f553a9c8751737c0968000a7ead47d5abad537652eb5c8
  Stored in directory: /root/.cache/pip/wheels/58/56/fe/2e93d842ffa9ea97746c1ab253d43502ed61c0689361a0224e
Successfully built pdfplumber
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is in

In [2]:
import pdfplumber
import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/redbull_model/

/content/gdrive/My Drive/redbull_model


In [5]:
REDBULL_INPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/redbull_input_folder/"
FUJITSU_INPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/fujitsu_input_folder/"
OUTPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/output_text/"

In [ ]:
for fl in os.listdir(INPUT_FOLDER):

  pdf = pdfplumber.open(INPUT_FOLDER+fl)
  pdf_text=pdf.pages[0].extract_text()
  with open(OUTPUT_FOLDER + fl.replace('.pdf','.txt'),'w') as f:
    f.write(pdf_text)

In [6]:
!pip install -U spacy -q
!python -m spacy info

     |████████████████████████████████| 6.0 MB 14.6 MB/s 
     |████████████████████████████████| 653 kB 64.8 MB/s 
     |████████████████████████████████| 181 kB 71.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 451 kB 77.6 MB/s 
     |████████████████████████████████| 10.1 MB 64.2 MB/s 

============================== Info about spaCy ==============================

spaCy version    3.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines                                      



In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [8]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("Skipping entity")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") 

100%|██████████| 885/885 [00:00<00:00, 2209.24it/s]


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-02-09 06:30:24,695] [INFO] Set up nlp object from config
[2022-02-09 06:30:24,710] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-02-09 06:30:24,715] [INFO] Created vocabulary
[2022-02-09 06:30:24,717] [INFO] Finished initializing nlp object
[2022-02-09 06:30:25,799] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.17    0.00    0.00    0.00    0.00
  2     200        113.33   1037.65   90.38   97.92   83.93    0.90
  6     400         26.38    126.31   91.67   85.94   98.21    0.92
 10     600         13.10    118.60   91.43   97

In [9]:
nlp_ner = spacy.load("/content/gdrive/MyDrive/redbull_model/model-best")

In [ ]:
doc =nlp_ner('''Bill From
TAX INVOICE
RedBull India Pvt. Ltd
c/o AWL India Pvt Ltd, Khasra no. 3401 JA, Invoice No: 4203304777
Village Amausi, Industrial Area,, Nadarganj, District, Uttar Invoice Date: 12-01-2022
Pradesh Ref. No. 01
Lucknow - 226008 Ref Date: 10.01.2022
State Code: 9 GSTIN:09AADCR5842H1Z6 Place of Supply: 9
Invoice To
Transporter: 09AAGCA4705P1ZB
G. A. ENTERPRISES
TransporterName: AWL INDIA PVT LTD
395/28, KASHMIRI MOHALLA, LUCKNOW
Vehicle No: UP32HN2010
Uttar Pradesh
LR no: LKO010005116
LUCKNOW - 226003
Mode of Transport:Road
State Code: 9 GSTIN:09AAVPH9194B1Z6
Delivery Address
G. A. ENTERPRISES
395/28, KASHMIRI MOHALLA, LUCKNOW
Uttar Pradesh
LUCKNOW - 226003
IRN: 694a04c9e4b526dabea9e77971f5f1bf0903adbe5bb543303f314dd701d67b9e
Rate Per
Serial No Material Description HSN/SAC Qty Gross Amt. Discount Amt. Net Amt. GST Rate % GST Tax Amt. Total Amt. (INR)
Item
IGST: 0.00
Red Bull IN Sugar Free 250ml WA
CGST: 1,163.00
RB205906 GST: 28
10 22029990 5 BOX 1,661.07 8,305.35 0.00 8,305.35 SGST/UGST: 11,628.35
Mfg. Dt.: 01.09.2021-5 CESS: 12
1,163.00
Batch No: 1957054
CESS: 997.00
IGST: 0.00
RB Red Edition IN Alu Can 24x250ml
CGST: 2,326.00
RB235674 GST: 28
20 22029990 10 BOX 1,661.07 16,610.70 0.00 16,610.70 SGST/UGST: 23,255.70
Mfg. Dt.: 26.10.2021-10 CESS: 12
2,326.00
Batch No: 1975317
CESS: 1,993.00
IGST: 0.00
Red Bull ED IN Alu Can 24x350ml
CGST: 1,466.00
RB201709 GST: 28
30 22029990 5 BOX 2,094.40 10,472.00 0.00 10,472.00 SGST/UGST: 14,661.00
Mfg. Dt.: 13.06.2021-4;28.04.2021-1 CESS: 12
1,466.00
Batch No: 1932313,1918841
CESS: 1,257.00
IGST: 0.00
Red Bull Energy Drink IN 250ml
CGST: 24,883.00
RB30925 GST: 28
40 22029990 107 BOX 1,661.07 177,734.49 0.00 177,734.49 SGST/UGST: 248,828.49
Mfg. Dt.: 20.09.2021-107 CESS: 12
24,883.00
Batch No: 1962068
CESS: 21,328.00
Total 213,122.54 0.00 213,122.54 85,251.00 298,373.54
TCS 298.38
Total Amount 298,671.92
Amount in words:INR Two Lakh Ninety Eight Thousand Six Hundred Seventy One and Ninety Two Paisa Only
Payment Terms - Due immediately
Declaration:
No credit of additional duty of custom (ADC) is admissible to the buyer. We declare that this document shows the actual price of the goods described and that all particular are true and
correct. 
We hereby certify that food/foods mentioned in this document is / are warranted to be of the nature and quality which it/these purports/purport to be.
No. of Box: 127
Gross weight - 866.760
FSSAI Lic. No - 12721066000743
For Red Bull India Pvt. Ltd.
Authorised signatory
Red Bull India Pvt. Ltd.
Registered Office Address: 1st Floor, B Wing, 215 Atrium, Kanakia Spaces, Chakala, Andheri Kurla Road, Andheri (E) Mumbai-400093. www.redbull.in
Tel:+91-22-66180888, Fax:+91-22-66180808, Email: info.in@redbull.com, CIN: U51909MH2007FTC169580 PAN: AADCR5842H''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [10]:
import pandas as pd

In [11]:
def extract_text_redbull_table(INPUT_FOLDER, output_csv):

  all_data =[]
  for fl in os.listdir(INPUT_FOLDER):
      print(fl)
      pdf_obj = pdfplumber.open(INPUT_FOLDER + fl)
      for page in pdf_obj.pages:
          tables = page.extract_tables()
          page_txt = page.extract_text()
          # print(page_txt)
          for table in tables:
              if 'INVOICE' in ''.join(table[0]):
                  invoice_no =  table[1][0].split('\n')[0].replace('Invoice No:','').strip()
                  print(invoice_no)
              if 'Serial No' in ''.join(table[0]) or 'Item' in ''.join(table[0]):
                  table_df = pd.DataFrame(table[1::],columns=table[0])
                  # print(table_df.columns)
                  for idx, row in table_df.iterrows():
                      if row['Serial No']: #.isnumeric():
                          # print(type(row['Serial No']))
  #                         print(idx)
                          if idx == 0:
                              doc = nlp_ner(page_txt)
                              all_label_text = []
                              for ent in doc.ents:
                                print(ent.label_)
                                if ent.label_ == "COMPANY NAME":
                                  company_name = ent.text
                                if ent.label_ == "INVOICE DATE":
                                  invoice_date = ent.text
                                if ent.label_ == "COMPANY GSTIN NUMBER":
                                  company_gstin = ent.text
                                if ent.label_ == "VEHICLE NO":
                                  vehicle_no = ent.text
                                if ent.label_ == "BUYER GSTIN NUMBER":
                                  buyer_gstin = ent.text
                                if ent.label_ == "CIN NO":
                                  cin_no = ent.text
                                if ent.label_ == "COMPANY PAN NO":
                                  company_pan = ent.text

                          tot_row = row['Serial No'] == 'Total'
                          if tot_row:

                            receipt_tot_amt = table_df[table_df['Serial No'] == 'Total Amount']['Total Amt. (INR)'].values[0]
                            # tcs =  table_df[table_df['Serial No'] == 'TCS']['Total Amt. (INR)'].values[0]
                            all_data.append([invoice_no,company_name,invoice_date,company_gstin,vehicle_no,buyer_gstin,cin_no,company_pan,receipt_tot_amt, row['Gross Amt.'], row['Discount Amt.'], row['Net Amt.'], row['GST Tax Amt.'], row['Total Amt. (INR)'], row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate Per\nItem'], row['Gross Amt.'], row['Discount Amt.'], row['Net Amt.'], row['GST Rate %'], row['GST Tax Amt.'],row['Total Amt. (INR)']])
                          else:
                              all_data.append([invoice_no,'','','','','','','','','','','','','', row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate Per\nItem'], row['Gross Amt.'], row['Discount Amt.'], row['Net Amt.'], row['GST Rate %'], row['GST Tax Amt.'], row['Total Amt. (INR)']  ])
                      
                  print("_________________________")
  out_df = pd.DataFrame(all_data, columns= ['Invoice No','COMPANY NAME','INVOICE DATE','COMPANY GSTIN NUMBER','VEHICLE NO','BUYER GSTIN NUMBER','CIN NO','COMPANY PAN NO', 'Invoice Total Amt', 'Total Gross Amt.', 'Total Discount Amt.', 'Total Net Amt.', 'Total GST Tac Amt.','Total Total Amt. (INR) ','Serial No', 'Material Description', 'HSN/SAC', 'Qty', 'Rate Per\nItem',
       'Gross Amt.', 'Discount Amt.', 'Net Amt.', 'GST Rate %', 'GST Tax Amt.',
       'Total Amt. (INR)'])
  
  out_df =out_df[~out_df['Serial No'].isin(['TCS', 'Total Amount'])]
  for inv_no in out_df['Invoice No'].unique():
    for col in ['Invoice Total Amt', 'Total Gross Amt.', 'Total Discount Amt.', 'Total Net Amt.', 'Total GST Tac Amt.','Total Total Amt. (INR) ','COMPANY NAME','INVOICE DATE','COMPANY GSTIN NUMBER','VEHICLE NO','BUYER GSTIN NUMBER','CIN NO','COMPANY PAN NO']:
        print(col)
        out_df.loc[((out_df['Invoice No'] == inv_no)), col] = out_df[((out_df['Invoice No'] == inv_no) & (out_df['Serial No'] == 'Total'))][col].values[0]

  out_df =out_df[~(out_df['Serial No'] == "Total")]   
  out_df.to_csv(OUTPUT_FOLDER+ output_csv)


In [19]:
extract_text_redbull_table(REDBULL_INPUT_FOLDER,'1023.csv')

MetroCash_CarryIndiaPvt.Ltd._4203304409.pdf
4203304409
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
MetroCash_CarryIndiaPvt.Ltd._4203304408.pdf
4203304408
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
MetroCash_CarryIndiaPvt.Ltd._4250010452.pdf
4250010452
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
MetroCash_CarryIndiaPvt.Ltd._4250010456.pdf
4250010456
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
MetroCash_CarryIndiaPvt.Ltd._4250010454.pdf
4250010454
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
MetroCash_CarryIndiaPvt.Ltd._4250010453.pdf
4250010453
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBE

In [13]:
def extract_text_fujitsu_table(INPUT_FOLDER, output_csv):
  all_data =[]
  for fl in os.listdir(INPUT_FOLDER):
      print(fl)
      pdf_obj = pdfplumber.open(INPUT_FOLDER + fl)
      tot_pages = len(pdf_obj.pages)
      for page_num, page in enumerate(pdf_obj.pages):
          tables = page.extract_tables()
  #         print(tables)
          invoice_no = ''
          invoice_date = ''
          for idx, row in pd.DataFrame(tables[0]).iterrows():
  #             print(row)
            
              if (('invoice' in ' '.join(row.fillna('').values).lower())  & ('date' in ' '.join(row.fillna('').values).lower())):
                  for spl in ' '.join(row.fillna('').values).split('\n'):
                    
                      if 'Invoice Number' in spl:
                          invoice_no = spl.replace('Invoice Number', '').replace(':', '').strip()
                      if 'Invoice Date' in spl:
                          invoice_date = spl.replace('Invoice Date', '').replace(':', '').strip()
                        
                    
              if (('ref' in ' '.join(row.fillna('').values).lower())  & ('date' in ' '.join(row.fillna('').values).lower())):
                  for spl in ' '.join(row.fillna('').values).split('\n'):
  #                     print(">>>>>>>",spl)
                      if 'Ref:' in spl:
                          invoice_no = spl.split('Ref:')[1].replace(':', '').strip()
                      if 'Date :' in spl:
                          invoice_date = spl.replace('Date :', '').replace(':', '').strip()
                    
            
              if (('item' in ' '.join(row.fillna('').values).lower()) & ('quantity' in ' '.join(row.fillna('').values).lower())):
                  hdr_idx = idx
            
                
            
                  df_ = pd.DataFrame(tables[0][hdr_idx+1::],columns=tables[0][hdr_idx])
        
                  tot_amt = ''
                    
                  amount = ''
                  quant = ''
                  unit_price = ''
                  hsn = ''
                  for idx, row in df_.fillna('').iterrows():

                      if 'Item\nNo' in df_.columns:
                          sl_no = row['Item\nNo']
                      if 'Sl.No.' in df_.columns:
                          sl_no = row['Sl.No.']
  #                     print(sl_no)
                      if sl_no.isnumeric():
                          sl_no_ = sl_no
                          if 'Quantity' in df_.columns:
                              quant = row['Quantity']
                          if 'Amount' in df_.columns:
                              amount = row['Amount']
                          if 'Unit Price' in df_.columns:
                              unit_price = row['Unit Price']
                          for col in df_.columns:
                              print(col, type(col))
                              if col != None:
  #                                 print('yo')
                  #                 continue
                                  if 'description' in col.lower():
                                      prod_desc = row[col]
                  #                     print(prod_desc)
                                      if 'unit\nprice' in col.lower():
                                          print('?????????',prod_desc)
                                          unit_price = prod_desc.split('\n')[0].split(' ')[-1]

                                      for splt_ in prod_desc.split('\n'):
  #                                         print(splt_)
                                          if 'HSN Code' in splt_:
                                              hsn = splt_.replace('HSN Code:','').strip()
                                          elif 'HSN' in splt_:
                                              hsn = splt_.replace('HSN:', '').strip()
                                          if 'VALUE IN INR :' in splt_:
  #                                             print('>>>>')
                                              tot_amt = splt_.replace('VALUE IN INR :', '').strip()

                      if row['Quantity'] == 'Total':
                          tot_amt = row['Amount']


                  all_data.append([invoice_no, invoice_date, tot_amt, sl_no_,prod_desc,hsn,unit_price, quant, amount])
  out_df = pd.DataFrame(all_data, columns = ['Invoice No', 'Invoice Date', 'Invoice Total Amount', 'Serial No', 'Material Description', 'HSN','Unit Price', 'Quantity', 'Amount'])
  out_df.to_csv(OUTPUT_FOLDER+ output_csv) 


In [16]:
extract_text_fujitsu_table(FUJITSU_INPUT_FOLDER, '102.csv')

HR0620000126_Integra_Delhi.pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000533.pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000534.pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000533 (1).pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000526.pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000524.pdf
Item
No <class 'str'>
Product Code & Description Uom <class 'str'>
Quantity <class 'str'>
Unit Price <class 'str'>
Amount <class 'str'>
MH2720000523 (3).pdf
Item
No <class 'str'>
Product Code & Descri